In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import wd

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [4]:
def gen_pattern_replace_and_matcher_for_SingleMLP(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = self.embed(x).view(-1,self.embed_output_dim)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      emb = self.embed(x)
      redency_part = emb[redency_part_slice] 
      unredency_part = emb[unredency_part_slice] 
      return self.redency_linear(redency_part.view(-1,self.redency_weight_len)) + self.unredency_linear(unredency_part.view(-1,self.unredency_weight_len))
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def workload_wdl(num_field, prefix,dim = 64,l = [1024,512,256]):
  print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(num_field)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(num_field)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_SingleMLP(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [6]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [8]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_wdl,l = [1024,512,256]):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True,
    profile_memory=True,  # 记录内存使用情况
    with_stack=True,  # 记录代码堆栈信息
    with_flops=True,  # 记录FLOPs
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log_wdl')  
                   
                   
                   
                   ) as prof:
        with record_function("model_inference"):
          for i in range(6):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
    return prof
  print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l)
  p1 = run(ori)
  p2 = run(modify)
  return p1, p2

In [9]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 755 / 1024 (73.7%)
Greatest absolute difference: 0.0036971718072891235 at index (845,) (up to 1e-05 allowed)
Greatest relative difference: 0.017008782817655797 at index (845,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:33 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:33 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:33 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of

(9.543021519978842, 100.91410330738249)


STAGE:2025-02-08 00:42:33 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:33 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:33 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(8.971293767293295, 0.04124163726575918)


(<torch.profiler.profiler.profile at 0x7fb99fbab2b0>,
 <torch.profiler.profiler.profile at 0x7fb99c6cd9d0>)

In [11]:
p1 , p2 = gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1524 / 2048 (74.4%)
Greatest absolute difference: 0.003769814968109131 at index (50,) (up to 1e-05 allowed)
Greatest relative difference: 0.01893820606099149 at index (1386,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:41 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:41 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:41 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of 

(8.427659670511881, 5.639346936290065)
(13.143817583719889, 2.482801646250563)


STAGE:2025-02-08 00:42:41 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:41 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:41 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [12]:
p1 , p2 = gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2957 / 4096 (72.2%)
Greatest absolute difference: 0.003923892974853516 at index (3874,) (up to 1e-05 allowed)
Greatest relative difference: 0.018893578132380424 at index (3102,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:44 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:44 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:44 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(13.558467229207357, 3.5603726978984525)
(17.09779103597005, 11.738536285419187)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3043 / 4096 (74.3%)
Greatest absolute difference: 0.00391232967376709 at index (550,) (up to 1e-05 allowed)
Greatest relative difference: 0.020351408582415832 at index (2913,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:44 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:44 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:44 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [13]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 752 / 1024 (73.4%)
Greatest absolute difference: 0.003878086805343628 at index (581,) (up to 1e-05 allowed)
Greatest relative difference: 0.01817190238426796 at index (1003,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:47 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:47 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:47 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of 

(6.037116050720215, 4.426647379318638)
(4.799962043762207, 3.113711343871728)


STAGE:2025-02-08 00:42:47 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb99fb77d30>,
 <torch.profiler.profiler.profile at 0x7fb990d0cb20>)

In [14]:
p1 , p2 = gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1513 / 2048 (73.9%)
Greatest absolute difference: 0.003528803586959839 at index (954,) (up to 1e-05 allowed)
Greatest relative difference: 0.0165988644955745 at index (1337,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:50 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:50 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:50 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of 

(7.5700680414835615, 2.7596285655641215)
(8.707324663798014, 3.75643746824355)


STAGE:2025-02-08 00:42:50 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:50 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [15]:
p1 , p2 = gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2921 / 4096 (71.3%)
Greatest absolute difference: 0.004261195659637451 at index (37,) (up to 1e-05 allowed)
Greatest relative difference: 0.02356401623822519 at index (4020,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:52 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:52 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:52 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(16.068061192830402, 7.176663144845306)
(19.360144933064777, 10.614682171055593)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2964 / 4096 (72.4%)
Greatest absolute difference: 0.004705548286437988 at index (3493,) (up to 1e-05 allowed)
Greatest relative difference: 0.018363238459456515 at index (1052,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:42:52 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:42:53 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:42:53 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [16]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 413 / 1024 (40.3%)
Greatest absolute difference: 0.003218412399291992 at index (937,) (up to 1e-05 allowed)
Greatest relative difference: 0.016248993997080415 at index (679,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:47:21 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:47:21 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:21 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of 

(9.044567743937174, 17.41763482085743)
(6.616830825805664, 2.8491358345187714)


STAGE:2025-02-08 00:47:21 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:21 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb9935aa610>,
 <torch.profiler.profiler.profile at 0x7fb98c56f7f0>)

In [17]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 724 / 2048 (35.4%)
Greatest absolute difference: 0.0038715600967407227 at index (1439,) (up to 1e-05 allowed)
Greatest relative difference: 0.012493453729462303 at index (1656,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:47:24 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:47:24 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:24 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.831204096476236, 1.9618118787750267)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 765 / 2048 (37.4%)
Greatest absolute difference: 0.003971755504608154 at index (760,) (up to 1e-05 allowed)
Greatest relative difference: 0.01664293198800859 at index (280,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:47:24 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:47:24 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:24 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(10.364611943562826, 2.466878691444031)


(<torch.profiler.profiler.profile at 0x7fb9934f96d0>,
 <torch.profiler.profiler.profile at 0x7fb8ac770970>)

In [18]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1563 / 4096 (38.2%)
Greatest absolute difference: 0.003816068172454834 at index (3759,) (up to 1e-05 allowed)
Greatest relative difference: 0.015480150635854815 at index (3007,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:47:27 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:47:27 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:27 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output 

(31.26684824625651, 9.634399109220087)
(26.73951784769694, 17.14288401267873)


STAGE:2025-02-08 00:47:27 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:27 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb98c551eb0>,
 <torch.profiler.profiler.profile at 0x7fb8ac75edc0>)

In [28]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 394 / 1024 (38.5%)
Greatest absolute difference: 0.0035179555416107178 at index (831,) (up to 1e-05 allowed)
Greatest relative difference: 0.014633362969100944 at index (486,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:18 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:18 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:18 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(11.69602076212565, 1.3334785206906317)
(9.795427322387695, 2.776508248795532)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 388 / 1024 (37.9%)
Greatest absolute difference: 0.004914581775665283 at index (521,) (up to 1e-05 allowed)
Greatest relative difference: 0.015744155166194356 at index (503,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:19 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:19 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:19 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb8694edfd0>,
 <torch.profiler.profiler.profile at 0x7fb86091d940>)

In [29]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 772 / 2048 (37.7%)
Greatest absolute difference: 0.004156053066253662 at index (1329,) (up to 1e-05 allowed)
Greatest relative difference: 0.016900440240300255 at index (73,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:21 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:21 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:21 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(22.871971130371094, 3.153439479319786)
(22.48358726501465, 17.15648572068555)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 735 / 2048 (35.9%)
Greatest absolute difference: 0.002621293067932129 at index (1239,) (up to 1e-05 allowed)
Greatest relative difference: 0.011953959408921877 at index (1910,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:21 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:21 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:21 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb999b6e670>,
 <torch.profiler.profiler.profile at 0x7fb8562edac0>)

In [30]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1549 / 4096 (37.8%)
Greatest absolute difference: 0.003972649574279785 at index (1154,) (up to 1e-05 allowed)
Greatest relative difference: 0.02094592282614747 at index (3267,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:23 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:23 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:23 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(45.67710558573405, 8.817776852588901)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1612 / 4096 (39.4%)
Greatest absolute difference: 0.004143804311752319 at index (1716,) (up to 1e-05 allowed)
Greatest relative difference: 0.020371523175025733 at index (1747,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:24 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(37.25353876749674, 11.891515283777052)


STAGE:2025-02-08 00:48:24 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:24 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb84beb4eb0>,
 <torch.profiler.profiler.profile at 0x7fb84bec2760>)

In [31]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 333 / 1024 (32.5%)
Greatest absolute difference: 0.005116701126098633 at index (121,) (up to 1e-05 allowed)
Greatest relative difference: 0.01696456676801084 at index (698,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:26 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:26 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:26 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(10.46450932820638, 1.198786580718863)
(6.599028905232747, 3.865389140224174)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 305 / 1024 (29.8%)
Greatest absolute difference: 0.0031543970108032227 at index (746,) (up to 1e-05 allowed)
Greatest relative difference: 0.012052457163492888 at index (346,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:26 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:27 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:27 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb845b1ab20>,
 <torch.profiler.profiler.profile at 0x7fb84bead070>)

In [32]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 642 / 2048 (31.3%)
Greatest absolute difference: 0.004233986139297485 at index (976,) (up to 1e-05 allowed)
Greatest relative difference: 0.017602428418968488 at index (708,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:29 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:29 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:29 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(19.863446553548176, 8.069236015570723)
(23.026506106058758, 3.1381625341787696)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 682 / 2048 (33.3%)
Greatest absolute difference: 0.004818975925445557 at index (1133,) (up to 1e-05 allowed)
Greatest relative difference: 0.013223447875541364 at index (1133,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:29 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:29 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:29 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb84583c2e0>,
 <torch.profiler.profiler.profile at 0x7fb8458da970>)

In [33]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1271 / 4096 (31.0%)
Greatest absolute difference: 0.003685295581817627 at index (2520,) (up to 1e-05 allowed)
Greatest relative difference: 0.018308183681217075 at index (1800,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:31 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:32 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:32 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(37.45841979980469, 5.8304921897918875)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1305 / 4096 (31.9%)
Greatest absolute difference: 0.0032805800437927246 at index (1933,) (up to 1e-05 allowed)
Greatest relative difference: 0.017524678551486296 at index (3844,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:32 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:32 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:32 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(29.34110164642334, 1.5757028440551342)


(<torch.profiler.profiler.profile at 0x7fb837374f70>,
 <torch.profiler.profiler.profile at 0x7fb837374fa0>)

In [34]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 298 / 1024 (29.1%)
Greatest absolute difference: 0.003897368907928467 at index (570,) (up to 1e-05 allowed)
Greatest relative difference: 0.016947029760423615 at index (325,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:36 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:36 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:36 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(21.79892857869466, 2.718651286512694)
(11.784116427103678, 4.031150532392126)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 334 / 1024 (32.6%)
Greatest absolute difference: 0.0037586092948913574 at index (832,) (up to 1e-05 allowed)
Greatest relative difference: 0.014827521154179981 at index (705,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:37 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:37 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:37 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb860ad5be0>,
 <torch.profiler.profiler.profile at 0x7fb845781220>)

In [35]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 647 / 2048 (31.6%)
Greatest absolute difference: 0.0031185150146484375 at index (1279,) (up to 1e-05 allowed)
Greatest relative difference: 0.01655498510000145 at index (1034,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:48:40 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:48:40 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:40 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output o

(34.41977500915527, 3.033463504683217)
(20.506660143534344, 1.6578859916510027)


STAGE:2025-02-08 00:48:40 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:48:40 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb83710f850>,
 <torch.profiler.profiler.profile at 0x7fb830ca4fa0>)

In [27]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1287 / 4096 (31.4%)
Greatest absolute difference: 0.0035219788551330566 at index (701,) (up to 1e-05 allowed)
Greatest relative difference: 0.018649896847960253 at index (1593,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:47:55 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 00:47:55 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:55 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(69.65216000874837, 5.302896410183975)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1296 / 4096 (31.6%)
Greatest absolute difference: 0.003970116376876831 at index (3939,) (up to 1e-05 allowed)
Greatest relative difference: 0.019779549549578793 at index (2725,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 00:47:56 3001906:3001906 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(38.399298985799156, 7.372280675907757)


STAGE:2025-02-08 00:47:56 3001906:3001906 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 00:47:56 3001906:3001906 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(<torch.profiler.profiler.profile at 0x7fb860913790>,
 <torch.profiler.profiler.profile at 0x7fb860a297c0>)